### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/dual_arm_cartesian/'

In [2]:
def get_random_joints(low_limit, up_limit):
    dof = len(low_limit)
    joint_angle = np.random.rand(dof)*(up_limit-low_limit) + low_limit     
    return joint_angle


### Setting the trajopt

In [3]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("robots/pr2-beta-static.zae")
env.Load("../env/bookshelves.env.xml")
env.SetDefaultViewer()

True

### Setting the robot

In [4]:
robot = env.GetRobots()[0]

left_arm = robot.GetManipulator("leftarm")
right_arm = robot.GetManipulator("rightarm")

robot.SetActiveDOFs(np.r_[right_arm.GetArmIndices(), left_arm.GetArmIndices()])
coeffs = [1,1,1,1,1,1,1,  1,1,1,1,1,1,1,] #coefficients for joint velocity cost

dof = len(robot.GetActiveDOFValues())
left_dof_away = np.array([ 0.5646,  1.1371, -0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
right_dof_away = np.array([ -0.5646,  1.1371, 0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
robot.SetDOFValues(left_dof_away, left_arm.GetArmIndices())
robot.SetDOFValues(right_dof_away, right_arm.GetArmIndices())
init_joint0 = robot.GetActiveDOFValues()

low_limit = robot.GetActiveDOFLimits()[0]
up_limit = robot.GetActiveDOFLimits()[1]
low_limit[4] = -np.pi
low_limit[6] = -np.pi
up_limit[4] = np.pi
up_limit[6] = np.pi


### Set the IK Solver 

In [5]:
hmat_target = np.array([[ 0.9689,  0.    , -0.2474,  0.4918],
       [-0.    ,  1.    ,  0.    ,  0.912 ],
       [ 0.2474, -0.    ,  0.9689,  0.8947],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])
quat_target = openravepy.quatFromRotationMatrix(hmat_target).tolist()
xyz_target = hmat_target[0:3,3].tolist()

# BEGIN IK for all manips
manips = [left_arm,right_arm]
for manip in manips:
    robot.SetActiveManipulator(manip)
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(
        robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        ikmodel.autogenerate()   
    
target_joint = ku.ik_for_link(hmat_target, left_arm, "l_gripper_tool_frame",
    filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
# END ik

### Construct a random problem 

### Build / Retrieve Dataset

In [6]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30,predictor=None):
    num_plan = 0
    comp_times = []
    costs = []
    tic = time.time()
    
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        robot.SetActiveDOFValues(init_joint0)
        target_joint_right,xyz_target_right,target_joint_left,xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        
        quat = [1,0,0,0]
        target_left = np.concatenate([xyz_target_left, quat])
        target_right = np.concatenate([xyz_target_right, quat])
        
        
        #Build memory using a set of predictors
        for func in predictor:
            #Predict the warmstart
            x_cur = np.concatenate([xyz_target_left,xyz_target_right])[None,:]
            if isinstance(func, Straight_Regressor):
                traj,_ = func.predict(init_joint0, target_joint)     
            else:
                traj,_ = func.predict(x_cur)
            traj = traj.reshape(-1,dof)
            traj[0,:] = init_joint0
            
            #Plan
            robot.SetActiveDOFValues(init_joint0)
            request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='pose')
            request_traj = add_constraint(request_traj, 'pose', 'l_gripper_tool_frame', target_left,n_steps-1)
            request_traj = add_constraint(request_traj, 'pose', 'r_gripper_tool_frame', target_right,n_steps-1)
            request_traj = set_init(request_traj, 'given_traj', traj)
            duration, result = run_opt(request_traj, env)    

            #Check traj result
            traj = result.GetTraj()
            if check_traj_pose(env,result, target_left, target_right):
                print 'Planning is successfull!'
                x_init.append(x_cur)
                y_init.append(traj.flatten())
                comp_times.append(duration)
                costs.append(result.GetCosts()[0][1])
                break
            else:
                print('Fail to find good solution!')
                continue   

        num_plan += 1
        
        
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [9]:
to_build_database = False
to_augment_database = False
num_traj = 1000
n_steps = 30
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps, predictor = [straight])
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)

### Reduce the data sample

In [11]:
x_init = x_init[0:1000]
y_init = y_init[0:1000]

### Apply Dimensionality Reduction to the data

In [12]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Build Function Approximator

In [13]:
dim_input = len(x_init[0])
is_load_regressor = False

straight = Straight_Regressor(dof = dof, n_steps=n_steps)
nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
bgmr = DP_GLM_Regressor()
bgmr_pca = DP_GLM_Regressor(is_transform='PCA')

if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    gpy_pca.load_from_file(FILENAME + 'gpy_pca')
    bgmr.load_from_file(FILENAME + 'bgmr')
    bgmr_pca.load_from_file(FILENAME + 'bgmr_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME + 'gpy_pca')

    print 'Planning for bgmr'
    bgmr.fit(x_init,y_init)
    bgmr.save_to_file(FILENAME + 'bgmr')

    print 'Planning for bgmr PCA'
    bgmr_pca.fit(x_init,y_init_reduced)
    bgmr_pca.save_to_file(FILENAME + 'bgmr_pca')

Planning for NN
Planning for GPY
Optimization restart 1/10, f = 334723.53256
Optimization restart 2/10, f = 334723.53248
Optimization restart 3/10, f = 334723.532475
Optimization restart 4/10, f = 334723.532531
Optimization restart 5/10, f = 334723.53249
Optimization restart 6/10, f = 334723.532608
Optimization restart 7/10, f = 334723.532651
Optimization restart 8/10, f = 334723.532622
Optimization restart 9/10, f = 334723.532469
Optimization restart 10/10, f = 334723.532533
Planning for GPY PCA
Optimization restart 1/10, f = 91942.9034261
Optimization restart 2/10, f = 91942.9034235
Optimization restart 3/10, f = 91942.9034361
Optimization restart 4/10, f = 91942.9034762
Optimization restart 5/10, f = 91942.9034283
Optimization restart 6/10, f = 91942.9034263
Optimization restart 7/10, f = 91942.9034273
Optimization restart 8/10, f = 91942.9034484
Optimization restart 9/10, f = 91942.9034246
Optimization restart 10/10, f = 91942.9034293
Planning for bgmr
Planning for bgmr PCA


### Creating the test cases

In [14]:
create_test = False
if create_test:
    x_test = []
    target_joints = []
    num_test = 500
    for i in range(num_test):
        robot.SetActiveDOFValues(init_joint0)
        target_joint_right,xyz_target_right,target_joint_left,xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        
        quat = [1,0,0,0]
        target_left = np.concatenate([xyz_target_left, quat])
        target_right = np.concatenate([xyz_target_right, quat])
 
        cur_x = (np.concatenate([xyz_target_left, xyz_target_right]))
        x_test.append(cur_x)
        target_joints.append(target_joint)

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    data_test['target_joints'] = target_joints
    test_file = open(FILENAME + 'data_test.npy', 'wb')
    pickle.dump( data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.npy', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    target_joints = data_test['target_joints']
    num_test = len(x_test)
    

In [15]:
def get_ik_both_arm(init_joint, x_cur, num_ik = 5):
    xyz_left = x_cur[0,:3].tolist()
    xyz_right = x_cur[0,3:].tolist()
    quat = [1,0,0,0]
    T_left = openravepy.matrixFromPose( np.r_[quat, xyz_left] )
    T_right = openravepy.matrixFromPose( np.r_[quat, xyz_right] )
    is_success = False
    
    #get a valid IK solution starting from a random configuration
    while is_success is False:
        random_joints = get_random_joints(low_limit, up_limit)
        #Find pose for left
        print 'Finding left'
        robot.SetActiveDOFValues(random_joints)
        robot.SetDOFValues(init_joint[:7], right_arm.GetArmIndices())

        joint_angle_left = ku.ik_for_link(T_left, left_arm, 'l_gripper_tool_frame',
          filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
        if joint_angle_left is None:
            print 'None'
            continue

        #check for collision
        robot.SetDOFValues(joint_angle_left, left_arm.GetArmIndices())
        if env.CheckCollision(robot) or robot.CheckSelfCollision(): continue
            
        #Find pose for right
        print 'Finding right'
        robot.SetActiveDOFValues(random_joints)
        robot.SetDOFValues(init_joint[7:], left_arm.GetArmIndices())
        joint_angle_right = ku.ik_for_link(T_right, right_arm, 'r_gripper_tool_frame',
          filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
        if joint_angle_right is None:continue

        #check for collision
        robot.SetDOFValues(joint_angle_right, right_arm.GetArmIndices())
        if env.CheckCollision(robot) or robot.CheckSelfCollision(): 
            print 'In collision'
            continue
            
        is_success = True
    
    
    #given the successful random_joints, get all valid IK solutions and shuffle them
    robot.SetActiveDOFValues(random_joints)
    robot.SetDOFValues(init_joint[:7], right_arm.GetArmIndices())

    joint_angle_lefts = ku.ik_for_link(T_left, left_arm, 'l_gripper_tool_frame',
      filter_options = openravepy.IkFilterOptions.CheckEnvCollisions, return_all_solns = True)
    
    robot.SetActiveDOFValues(random_joints)
    robot.SetDOFValues(init_joint[7:], left_arm.GetArmIndices())
    joint_angle_rights = ku.ik_for_link(T_right, right_arm, 'r_gripper_tool_frame',
          filter_options = openravepy.IkFilterOptions.CheckEnvCollisions, return_all_solns = True)    
    
    joint_angles = [np.concatenate([joint_angle_right,joint_angle_left])]
    
    j = 0
    while len(joint_angles) < num_ik and j < 30:
        if len(joint_angle_lefts) == 0 or len(joint_angle_rights) == 0:
            break
        np.random.shuffle(joint_angle_lefts)
        np.random.shuffle(joint_angle_rights)
        joint_angle_left = joint_angle_lefts[0]
        joint_angle_right = joint_angle_rights[0]
        joint_angle = np.concatenate([joint_angle_right,joint_angle_left])
        robot.SetActiveDOFValues(joint_angle)
        j += 1
        if env.CheckCollision(robot) or robot.CheckSelfCollision(): 
            continue
        else:
            joint_angles += [joint_angle]
            
    return joint_angles

In [16]:
def compute_costs(traj):
    cost = 0
    for i in range(len(traj)-1):
        cost += np.linalg.norm(traj[i]-traj[i+1])
    return cost

In [17]:
#load gpy for joint angle
FILENAME_AUX = 'data/dual_arm_fixed_init/'
gpy_aux = GPy_Regressor(dim_input = 14)
gpy_aux.load_from_file(FILENAME_AUX + 'gpy')

In [18]:
#load gpy and nn for cost
FILENAME_AUX = 'data/dual_arm_fixed_init/'
gpy_cost = GPy_Regressor(dim_input = 14)
gpy_cost.load_from_file(FILENAME_AUX + 'gpy_cost')

nn_cost = NN_Regressor()
nn_cost.load_from_file(FILENAME_AUX + 'nn_cost')

In [20]:
method_names = ['STD    ',  'NN     ', 'GPY    ', 'GPY_PCA',  'BGMR','BGMR_PCA', 'METRIC GPY_PCA','METRIC DIRECT']
methods = [straight, nn, gpy,gpy_pca, bgmr, bgmr_pca]#
results = dict()
result,success, duration = dict(),dict(),dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 250
ik_times = []
quat = [1,0,0,0]
for i in range(num_test):
    print('{}th Planning'.format(i))
    
    #setting up the problem case
    #index = np.random.randint(0,len(x_test))
    index = i
    x_cur = x_test[index:index+1,:]
    target_joint = target_joints[index]
    robot.SetActiveDOFValues(init_joint0)
    
    #Other warmstart methods
    target_left = np.concatenate([x_cur[0,0:3], quat])
    target_right = np.concatenate([x_cur[0,3:], quat])
    
    request_traj = define_request(coeffs = coeffs,init_type='given_traj',constraint_type='pose')
    request_traj = add_constraint(request_traj, 'pose', 'l_gripper_tool_frame', target_left,n_steps-1)
    request_traj = add_constraint(request_traj, 'pose', 'r_gripper_tool_frame', target_right,n_steps-1)
    
    
    for i,method in enumerate(methods):
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        elif isinstance(method, DP_GLM_Regressor):
            traj,cov = method.predict(x_cur, return_gmm = True)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = y_pca.inverse_transform(traj)
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint0
        robot.SetActiveDOFValues(init_joint0)
        request_traj = set_init(request_traj, 'given_traj', traj)
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj_pose(env,result[method_name], target_left, target_right)

    
    #run GPY_aux with joint planning
    method_name = method_names[-2]
    num_ik = 5
    trajs = []
    targets = []
    costs = []
    tic = time.time()
    robot.SetActiveDOFValues(target_joint)
    joint_angles = get_ik_both_arm(init_joint0,x_cur, num_ik)
    joint_angles.append(target_joint)
    for j in range(len(joint_angles)):
        joint_angle = joint_angles[j]
        traj,cov = gpy_aux.predict(joint_angle[None,:])
        traj = traj.reshape(-1,dof)
        trajs.append(traj)
        targets.append(joint_angle)
        costs.append(compute_costs(traj))
    toc = time.time()
    print toc-tic
    ik_times.append(toc-tic)
    min_index = np.argmin(costs)
    print('Min index based on init_guess: {}'.format(min_index))
    print costs
    
    traj = trajs[min_index]
    target_chosen = targets[min_index]
    request_traj_aux = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj_aux = add_constraint(request_traj_aux, 'joint', '', target_chosen,-1)
    
    traj[0,:] = init_joint0
    robot.SetActiveDOFValues(init_joint0)
    request_traj_aux = set_init(request_traj_aux, 'given_traj', traj)
    duration[method_name], result[method_name] = run_opt(request_traj_aux, env)    
    success[method_name] = check_traj(env,result[method_name], target_chosen)

    #run GPy_cost with joint planning
    method_name = method_names[-1]
    num_ik = 5
    trajs = []
    targets = []
    costs = []
    for j in range(len(joint_angles)):
        joint_angle = joint_angles[j]
        cost,_ = gpy_cost.predict(joint_angle[None,:])
        costs.append(cost)
    min_index = np.argmin(costs)
    print('Min index based on cost: {}'.format(min_index))
    print costs
    joint_angle = joint_angles[min_index]
    traj,cov = gpy_aux.predict(joint_angle[None,:])
    traj = traj.reshape(-1,dof)
    target_chosen = joint_angle
    request_traj_aux = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj_aux = add_constraint(request_traj_aux, 'joint', '', target_chosen,-1)
    
    traj[0,:] = init_joint0
    robot.SetActiveDOFValues(init_joint0)
    request_traj_aux = set_init(request_traj_aux, 'given_traj', traj)
    duration[method_name], result[method_name] = run_opt(request_traj_aux, env)    
    success[method_name] = check_traj(env,result[method_name], target_chosen)
    
    
    
    #Record the result
    for method_name in method_names:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])


0th Planning
Optimization is completed in 0.287973165512 s!
Optimization is completed in 0.459728002548 s!
Optimization is completed in 0.748448848724 s!
Optimization is completed in 0.817936897278 s!
Optimization is completed in 0.593877792358 s!
Optimization is completed in 0.656775951385 s!
Finding left
Finding right
0.0990779399872
Min index based on init_guess: 4
[9.8766973090083834, 8.8118508529292896, 7.2159832624021636, 8.669300502715771, 7.1637229815278189, 7.2492712519353795]
Optimization is completed in 0.202389001846 s!
Min index based on cost: 4
[array([[ 3.2086]]), array([[ 2.6226]]), array([[ 1.7838]]), array([[ 2.5074]]), array([[ 1.7681]]), array([[ 1.7943]])]
Optimization is completed in 0.202148914337 s!
1th Planning
Optimization is completed in 1.34830188751 s!
Optimization is completed in 1.45098495483 s!
Optimization is completed in 5.13986301422 s!
Optimization is completed in 1.44959688187 s!
Optimization is completed in 1.37908697128 s!
Optimization is complete

### Printing Results

In [21]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
for method in method_names:
    successes = np.array(results[method]['successes'])
    success = np.count_nonzero(successes)
    
    comp_times = np.array(results[method]['comp_times'])[successes]
    costs = np.array(results[method]['costs'])[successes]
    func_evals = np.array(results[method]['func_evals'])[successes]
    qp_solves = np.array(results[method]['qp_solves'])[successes]
    
    print('{0}: \t& {1:.3f} \t& {2:.2f} \t& {3:.3f}  \\\\'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success))
    

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD    : 	& 0.656 	& 1.10 	& 1.849  \\
NN     : 	& 0.736 	& 1.38 	& 1.838  \\
GPY    : 	& 0.664 	& 1.90 	& 1.865  \\
GPY_PCA: 	& 0.672 	& 1.80 	& 1.780  \\
BGMR: 	& 0.744 	& 1.48 	& 1.816  \\
BGMR_PCA: 	& 0.756 	& 1.46 	& 1.843  \\
METRIC GPY_PCA: 	& 0.880 	& 0.73 	& 1.484  \\
METRIC DIRECT: 	& 0.876 	& 0.76 	& 1.478  \\


In [60]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
for method in method_names:
    successes = np.array(results[method]['successes'])
    success = np.count_nonzero(successes)
    
    comp_times = np.array(results[method]['comp_times'])[successes]
    costs = np.array(results[method]['costs'])[successes]
    func_evals = np.array(results[method]['func_evals'])[successes]
    qp_solves = np.array(results[method]['qp_solves'])[successes]
    
    print('{0}: \t& {1:.3f} \t& {2:.2f} \t& {3:.3f}  \\\\'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success))
    

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD    : 	& 0.767 	& 1.29 	& 1.958  \\
NN     : 	& 0.800 	& 1.41 	& 1.925  \\
GPY    : 	& 0.800 	& 1.84 	& 1.991  \\
GPY_PCA: 	& 0.633 	& 2.00 	& 2.154  \\
BGMR: 	& 0.700 	& 1.59 	& 2.160  \\
BGMR_PCA: 	& 0.800 	& 1.68 	& 2.211  \\
METRIC GPY_PCA: 	& 0.833 	& 0.65 	& 1.441  \\
METRIC DIRECT: 	& 0.800 	& 0.68 	& 1.438  \\


### Save Result

In [23]:
pickle.dump(results,open(FILENAME + 'result.pkl', 'wb') )

### Load Result 

In [ ]:
results = pickle.load(open(FILENAME + 'result_both.pkl', 'rb'))

### Printing Results Tied to Standard Success

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    func_evals = np.array(results[method]['func_evals'])[standard_successes][successes]
    qp_solves = np.array(results[method]['qp_solves'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    